# Calculating Fires Area (FIRMS)

In [1]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

## Loading, Clipping, and Masking Imagery

In [ ]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_ideam = ee.Image("projects/ee-juamiji/assets/SouthAmerica_2001_primary").selfMask()
primaryForest_ideam_COL = primaryForest_ideam.clip(colombiaMpios)

# Import the Imagery of Nighttime Lights (Harmonized DMSP-OLS and VIIRS)
firms = ee.ImageCollection("FIRMS")

# Function to mask and binarize fire pixels
def binarize_fire_pixels(img):
    fire_mask = img.select('T21').mask()  # Mask where T21 exists (fire pixels)
    binary_img = ee.Image(1).updateMask(fire_mask)  # Replace all valid T21 pixels with 1
    return binary_img.copyProperties(img, img.propertyNames())  # Keep original metadata

# Apply to the FIRMS collection and clip to Colombia
firms_binary = firms.map(binarize_fire_pixels).map(lambda img: img.clip(colombiaMpios))

# Clip the NL images to Colombia's boundaries
firms_COL = firms_binary.map(lambda image: image.clip(colombiaMpios))

# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_ideam_COL.eq(1)

# Masking the firms imagery using the primary mask (cover equals 1)
firms_COLmasked = firms_COL.map(lambda img: img.updateMask(primaryMask))

## Code for all years from 2000 to 2021

In [4]:
# Calculate the total loss area per municipality
def calculate_loss_area(feature, year):

    firms_year = firms_COLmasked.filterDate(f'{year}-01-01', f'{year}-12-31').max()

    # Calculate the loss area for the current year in square meters
    firms_year_area = firms_year.multiply(ee.Image.pixelArea())

    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    fire_area = firms_year_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('constant')  # Replace 'lossyear' with the correct property name if different
    
    # Dynamically name the loss area property
    fire_area_property_name = f'firearea{year}'

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        fire_area_property_name: fire_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
year_results_dict = {}

for year in range(2000, 2021):
    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_loss_area(feature, year))
    year_results_dict[f'fire_year_results{year}'] = current_year_results
    # Optionally print the first feature's info to check
    print(year_results_dict[f'fire_year_results{year}'].first().getInfo())


{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2000': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2001': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2002': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2003': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2004': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2005': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2006': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'firearea2007': 0}}
{'type': 'Feature', 'geometry': None, 'i

## Exporting CVS files to my Google Drive  
### NOTE: see the progress of exports at [GEE Task Manager](https://code.earthengine.google.com/tasks)

In [6]:
# Define a function to export FeatureCollections to Google Drive
def export_to_drive(feature_collection, year, description_prefix="FiresData", folder_name="EarthEngineExports"):
    # Create the export task
    export_task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=f"{description_prefix}_Year{year}",
        folder="EarthEngineExports",
        fileNamePrefix=f"Fires_Year{year}",
        fileFormat="CSV"
    )
    # Start the export task
    export_task.start()
    print(f"Exporting {description_prefix}_Year{year} to Google Drive...")

# Loop through each item in the dictionary and initiate an export
for year, feature_collection in year_results_dict.items():
    # Extract the year from the key (which is a string like 'year_results11')
    extracted_year = int(year.replace('fire_year_results', ''))
    export_to_drive(feature_collection, extracted_year)
    
#END

Exporting FiresData_Year2000 to Google Drive...
Exporting FiresData_Year2001 to Google Drive...
Exporting FiresData_Year2002 to Google Drive...
Exporting FiresData_Year2003 to Google Drive...
Exporting FiresData_Year2004 to Google Drive...
Exporting FiresData_Year2005 to Google Drive...
Exporting FiresData_Year2006 to Google Drive...
Exporting FiresData_Year2007 to Google Drive...
Exporting FiresData_Year2008 to Google Drive...
Exporting FiresData_Year2009 to Google Drive...
Exporting FiresData_Year2010 to Google Drive...
Exporting FiresData_Year2011 to Google Drive...
Exporting FiresData_Year2012 to Google Drive...
Exporting FiresData_Year2013 to Google Drive...
Exporting FiresData_Year2014 to Google Drive...
Exporting FiresData_Year2015 to Google Drive...
Exporting FiresData_Year2016 to Google Drive...
Exporting FiresData_Year2017 to Google Drive...
Exporting FiresData_Year2018 to Google Drive...
Exporting FiresData_Year2019 to Google Drive...
Exporting FiresData_Year2020 to Google D